In [ ]:
%pip install -U scikit-learn

In [26]:
import pandas as pd
df = pd.read_csv('/kaggle/input/regression-with-an-insurance-dataset-02/02_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [27]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost

voting_reg = VotingRegressor(
    estimators=[
        ('lin_reg', LinearRegression()),
        ('xgboost_01', xgb.XGBRegressor(max_depth=6, n_estimators=40, random_state=42)),
        ('xgboost_02', xgb.XGBRegressor(max_depth=8, n_estimators=40, random_state=42)),
        ('catboost_01',Catboost(iterations=10, depth=8, learning_rate=1, random_state=42)),
        ('catboost_02',Catboost(iterations=20, depth=8, learning_rate=1, random_state=42)),
    ]
)
voting_reg.fit(X_train, y_train)

0:	learn: 1.0692702	total: 120ms	remaining: 1.08s
1:	learn: 1.0629547	total: 224ms	remaining: 896ms
2:	learn: 1.0623935	total: 316ms	remaining: 738ms
3:	learn: 1.0620984	total: 411ms	remaining: 617ms
4:	learn: 1.0617498	total: 511ms	remaining: 511ms
5:	learn: 1.0615620	total: 615ms	remaining: 410ms
6:	learn: 1.0613211	total: 716ms	remaining: 307ms
7:	learn: 1.0611418	total: 824ms	remaining: 206ms
8:	learn: 1.0609993	total: 931ms	remaining: 103ms
9:	learn: 1.0608489	total: 1.03s	remaining: 0us
0:	learn: 1.0692702	total: 126ms	remaining: 2.4s
1:	learn: 1.0629547	total: 232ms	remaining: 2.08s
2:	learn: 1.0623935	total: 324ms	remaining: 1.84s
3:	learn: 1.0620984	total: 419ms	remaining: 1.68s
4:	learn: 1.0617498	total: 519ms	remaining: 1.56s
5:	learn: 1.0615620	total: 616ms	remaining: 1.44s
6:	learn: 1.0613211	total: 718ms	remaining: 1.33s
7:	learn: 1.0611418	total: 824ms	remaining: 1.24s
8:	learn: 1.0609993	total: 929ms	remaining: 1.13s
9:	learn: 1.0608489	total: 1.03s	remaining: 1.03s
10:

VotingRegressor(estimators=[('lin_reg', LinearRegression()),
                            ('xgboost_01',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          inte...
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=8,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=40,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=42, ...)),
                            ('catboost_01',
                             <catboost.core.CatBoostRegressor object at 0x7d4a866c1000>),
                            ('catboost_02',
                             <catboost.core.CatBoostRegressor object at 0x7d4a866c1030>)])

In [28]:
from sklearn.metrics import root_mean_squared_error
y_pred = voting_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0565059777348034

In [29]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [30]:
y_pred = voting_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.063788928622013

In [31]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/kaggle/input/regression-with-an-insurance-dataset-02/02_test.csv')
del test_set['Premium Amount']
y_pred = voting_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/kaggle/input/regression-with-an-insurance-dataset-02/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('voting_reg_03.csv', index=False)